# TCC - Previsão de vendas usando LightGBM

In [ ]:
## Parameters

lags = list(range(1, 8)) + [14, 21, 28]
windows = [7, 30]


index = ['id', 'd', 'date']

categorical_features = [
    'wday',
    'month'
]

string_features = [
    'cat_id',
    'dept_id',
    'item_id',
    'state_id',
    'store_id',
    'year'
]

indexed_features = [f + '_index' for f in string_features]

numerical_features = [
    'sell_price',
    *[f'sales_lag_{d}' for d in lags],
    *[f'sales_lag_sum_{d}' for d in windows],
    *[f'sales_lag_mean_{d}' for d in windows]
]

features = indexed_features + categorical_features + numerical_features

target = ['sales']

output_cols = index + features + target

## Read and join data

In [ ]:
import os
import pandas as pd
from pathlib import Path

pd.set_option('display.max_columns', None)

In [ ]:
from pyspark.sql.context import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window as w

spark = (
    SparkSession.builder
    .config("spark.driver.memory", "16g")
    .config("spark.driver.cores", "16")
    .config("spark.driver.maxResultSize", "0")
    .config('spark.sql.execution.arrow.pyspark.enabled', 'true')
    .getOrCreate()
)

import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)

In [ ]:
src_path = Path('../source_data/')

calendar = ps.read_csv(str(src_path / 'calendar.csv'))
prices = ps.read_csv(str(src_path / 'sell_prices.csv'))
train = ps.read_csv(str(src_path / 'sales_train_validation.csv'))

In [ ]:
calendar.head()

In [ ]:
prices.head()

In [ ]:
train.head()

In [ ]:
len(train)

In [ ]:
df = (
    train
    .melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='sales')
    .spark.repartition(64)
    .merge(calendar, on='d')
    .merge(prices, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')
    .sort_values(['state_id', 'store_id', 'dept_id', 'cat_id', 'item_id', 'date', 'id'])
)

In [ ]:
df.to_parquet('data_joined')

## Prepare features

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
df = spark.read.parquet('data_joined')

### Fill missing prices

In [ ]:
from pyspark.sql.window import Window as w

In [ ]:
df.limit(5).toPandas()

In [ ]:
window_price = w.partitionBy('store_id', 'item_id').orderBy('date')

In [ ]:
df = df.withColumn('sell_price', f.last('sell_price', ignorenulls=True).over(window_price))

### Encode string features

In [ ]:
indexer = StringIndexer(inputCols=string_features, outputCols=indexed_features, handleInvalid='keep')

In [ ]:
df = indexer.fit(df).transform(df)

### Create lags

In [ ]:
lag_window = w.partitionBy('id').orderBy('date')

In [ ]:
for days in lags:
    df = df.withColumn(f'sales_lag_{days}', f.lag('sales', days).over(lag_window))
    # df[f'lag_{days}'] = df.groupby('id')['sales'].shift(days)

### Create rolling metrics

In [ ]:
for days in windows:
    df = df.withColumn(f'sales_lag_sum_{days}', f.sum('sales').over(lag_window.rowsBetween(-31, -1)))
    df = df.withColumn(f'sales_lag_mean_{days}', f.mean('sales').over(lag_window.rowsBetween(-31, -1)))
    # df[f'lag_sum_{days}'] = df.groupby('id').rolling(days).sum()
    # df[f'lag_mean_{days}'] = df.groupby('id').rolling(days).mean()

In [ ]:
df.filter(f.col('id') == 'FOODS_1_002_CA_3_validation').toPandas().to_csv('sample.csv', index=False)

In [ ]:
df = df.select(output_cols)

In [ ]:
for col, dtype in df.dtypes:
    if dtype == 'double':
        df = df.withColumn(col, f.col(col).cast('float'))

In [ ]:
df.write.mode('overwrite').parquet('data_preprocessed')

In [ ]:
df.select(f.min('date'), f.max('date')).show()

In [ ]:
df.limit(5).toPandas()

## Training

In [ ]:
import lightgbm as lgbm
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_parquet('data_preprocessed')

In [ ]:
for col, dtype in df.dtypes.reset_index().to_numpy():
    if dtype == 'float64':
        df[col] = df[col].astype('float32')

In [ ]:
df_train = df[df['date'] < '2015-01-01']
df_test = df[df['date'] >= '2015-01-01']

In [ ]:
x_train = df_train[features]
y_train = df_train[target]

In [ ]:
x_test = df_test[features]
y_test = df_test[target]

#### LGBM

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mse',
    'num_iterations': 100,
    'verbose': -1
}

model = lgbm.LGBMRegressor(**params)

In [ ]:
best = model.fit(x_train, y_train, categorical_feature=categorical_features + indexed_features, feature_name=features)

In [ ]:
model.fitted_

In [ ]:
best.booster_.save_model('model.txt')

#### Linear Regression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(x_train.fillna(0), y_train)

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

### Results

In [ ]:
from sklearn.metrics import mean_squared_error as mse

In [ ]:
shape = (len(y_test), )

In [ ]:
y_pred = best.predict(x_test).reshape(shape)
y_pred_rounded = y_pred.round()

In [ ]:
y_pred_lr = lr.predict(x_test.fillna(0)).reshape(shape)
y_pred_rounded_lr = y_pred_lr.round()

In [ ]:
y_true = y_test.to_numpy().reshape(shape)

In [ ]:
y_dummy_lag_1 = np.array(df_test['sales_lag_1'].fillna(0)).reshape(shape)

In [ ]:
y_pred

In [ ]:
y_pred_lr

In [ ]:
y_true

In [ ]:
y_dummy_lag_1

In [ ]:
def wape(y_true, y_pred):
    y_t = np.array(y_true)
    y_p = np.array(y_pred)
    return np.sum(np.abs(y_t - y_p)) / np.sum(y_t)

In [ ]:
def wpe(y_true, y_pred):
    y_t = np.array(y_true)
    y_p = np.array(y_pred)
    return np.sum(y_t - y_p) / np.sum(y_t)

In [ ]:
def get_results(y_true, y_pred, precision=3):
    res = {
        'mse': round(mse(y_true, y_pred), precision),
        'wape': round(wape(y_true, y_pred), precision),
        'wpe': round(wpe(y_true, y_pred), precision)
    }
    
    return res

In [ ]:
results = [
    {
        'Decimal': get_results(y_true, y_pred),
        'Arredondado': get_results(y_true, y_pred_rounded)
    }
]

In [ ]:
get_results(y_true, y_pred)

In [ ]:
get_results(y_true, y_pred_rounded)

In [ ]:
get_results(y_true, y_pred_lr)

In [ ]:
get_results(y_true, y_pred_rounded_lr)

In [ ]:
get_results(y_true, y_dummy_lag_1)

In [ ]:
latex_out = pd.read_excel('results.xlsx', header=[0,1], index_col=0).to_latex()

#### Feature Importances

In [ ]:
best = lgbm.Booster(model_file='model.txt')

In [ ]:
lgbm.plot_importance(best, height=0.3, grid=False, figsize=(10, 6))